# Uittesten van strategie elementen in Pandas voor CryptoMaven bot. 


## Buy the dip

Deze strategie is gebaseert een RSI en MFI die tegelijkertijd op het laagste punt zijn gekomen. Als dit punt ook nog eens 'bounced' van een supportniveau, dan is de kans groot dat er een koersstijging gaat plaatsvinden.

De volgende indicatoren worden hierbij gebruikt:
* CPR - Pivots voor het vinden van vooraf berekende support/resistance niveaus
* RSI10 - om oversold signalen te krijgen
* MFI10 - om oversold signalen te krijgen
* Wavetrend oscillator - Als RSI/MF oversold signalen ook in confluence zijn met oversold wavetrend, dan instappen

Voor instappen heb ik een aantal regels opgesteld:
* Instappen na de derde groene candle
* Instappen bij wavetrend signaal

Overige regels zijn:
* Stoploss is 20 pips onder de laagste wick van het oversold signaal.
* TP1 is een R:R van 1:1 (mogelijk naar 1:2 of hoger bij meer ervaring, maar niet te hoog ivm. target niet halen)
* Bij halen target 50% winst nemen en SL naar break even zetten. 
* Trend uitzitten tot het aflopen van RSI/MF oversold signaal dan 25% uitstappen (TP2)
* TSL verzetten naar 50% afstand tussen TP1 en TP2 (= TP3)
* Uitstappen indien TP3 wordt geraakt bij retracement OF indien prijs verder stijgt bij volgende oversold RSI/MFI signaal (zoals TP2)

In het begin traden op Spot of met 2x leverage. Indien dit een vertrouwde strategie wordt, dan leverage langzaam ophogen naar 5x max (ivm grotere verliesrisico's)

**ALTIJD TRADEN OBV. RISK & MONEY MANAGEMENT**

![CryptoMaven2 Chart](./images/buy-the-dip.png)

## Data in dataframe laden en controleren op juistheid

Data wordt dmv. een python3 functie gedownload van Binance.

In [6]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("./BTCUSDT_1h.csv")

# Reading columns
df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore'],
      dtype='object')

In [7]:
df.shape

(500, 12)

In [8]:
df.info

<bound method DataFrame.info of         # openTime      open      high       low     close       volume  \
0    1603220400000  11964.58  11981.20  11881.00  11912.35  2744.144578   
1    1603224000000  11912.42  11912.79  11855.00  11897.92  2523.731342   
2    1603227600000  11897.92  11937.73  11877.49  11922.53  1409.942909   
3    1603231200000  11922.53  11961.00  11918.22  11950.66  1270.092250   
4    1603234800000  11950.66  11961.16  11885.00  11909.99  1551.691786   
..             ...       ...       ...       ...       ...          ...   
495  1605002400000  15383.84  15430.00  15302.58  15305.14  2365.383922   
496  1605006000000  15305.15  15323.67  15155.00  15279.24  4014.340547   
497  1605009600000  15279.24  15393.65  15213.00  15287.61  3484.423100   
498  1605013200000  15287.61  15330.00  15179.00  15306.83  3370.999279   
499  1605016800000  15306.84  15308.04  15238.01  15239.49   882.070653   

         closeTime  qouteAssetVolume  numberOfTrades  takerBuyBaseA

In [9]:
df.dtypes

# openTime                    int64
open                        float64
high                        float64
low                         float64
close                       float64
volume                      float64
closeTime                     int64
qouteAssetVolume            float64
numberOfTrades                int64
takerBuyBaseAssetVolume     float64
takerBuyQuoteAssetVolume    float64
ignore                        int64
dtype: object

In [10]:
# Aanpassen van unix time naar human time
df['# openTime']=(pd.to_datetime(df['# openTime'],unit='ms'))
df['closeTime']=(pd.to_datetime(df['closeTime'],unit='ms'))

In [11]:
# Watch last 10 entries
df[['# openTime', 'open', 'high', 'low', 'close', 'volume']].tail(10)

,# openTime,open,high,low,close,volume
490,2020-11-10 05:00:00,15387.74,15441.18,15308.21,15357.92,2178.822453
491,2020-11-10 06:00:00,15357.92,15390.18,15247.21,15275.82,2558.920212
492,2020-11-10 07:00:00,15275.83,15333.49,15250.47,15274.74,1981.500383
493,2020-11-10 08:00:00,15274.75,15460.00,15261.01,15371.37,3795.218353
494,2020-11-10 09:00:00,15371.36,15440.00,15367.40,15383.84,2253.766369
495,2020-11-10 10:00:00,15383.84,15430.00,15302.58,15305.14,2365.383922
496,2020-11-10 11:00:00,15305.15,15323.67,15155.00,15279.24,4014.340547
497,2020-11-10 12:00:00,15279.24,15393.65,15213.00,15287.61,3484.423100
498,2020-11-10 13:00:00,15287.61,15330.00,15179.00,15306.83,3370.999279
499,2020-11-10 14:00:00,15306.84,15308.04,15238.01,15239.49,882.070653


# Pandas_ta indicatoren

Hier worden alleen RSI10 en MFI10 gebruikt voor signalering

In [12]:
df.ta.rsi(length=10, append=True)
df.ta.mfi(length=10, append=True)

df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'RSI_10', 'MFI_10'],
      dtype='object')

In [13]:
# waardes afronden op 2 cijfers achter de komma
df.round(2)

,# openTime,open,high,low,close,volume,closeTime,qouteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,takerBuyQuoteAssetVolume,ignore,RSI_10,MFI_10
0,2020-10-20 19:00:00,11964.58,11981.20,11881.00,11912.35,2744.14,2020-10-20 19:59:59.999,32742666.69,39342,1126.58,13442658.74,0,NaN,NaN
1,2020-10-20 20:00:00,11912.42,11912.79,11855.00,11897.92,2523.73,2020-10-20 20:59:59.999,29989751.46,39583,1251.68,14872868.67,0,0.00,NaN
2,2020-10-20 21:00:00,11897.92,11937.73,11877.49,11922.53,1409.94,2020-10-20 21:59:59.999,16790213.05,28867,642.19,7647593.84,0,65.46,NaN
3,2020-10-20 22:00:00,11922.53,11961.00,11918.22,11950.66,1270.09,2020-10-20 22:59:59.999,15160673.38,27532,603.06,7199003.46,0,81.14,NaN
4,2020-10-20 23:00:00,11950.66,11961.16,11885.00,11909.99,1551.69,2020-10-20 23:59:59.999,18500404.33,24948,757.03,9026310.96,0,46.92,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-11-10 10:00:00,15383.84,15430.00,15302.58,15305.14,2365.38,2020-11-10 10:59:59.999,36378608.11,48025,1116.96,17178903.67,0,47.59,52.94
496,2020-11-10 11:00:00,15305.15,15323.67,15155.00,15279.24,4014.34,2020-11-10 11:59:59.999,61214714.46,65654,1954.02,29797695.54,0,45.54,40.16
497,2020-11-10 12:00:00,15279.24,15393.65,15213.00,15287.61,3484.42,2020-11-10 12:59:59.999,53310596.35,56464,1696.47,25952126.02,0,46.37,43.31
498,2020-11-10 13:00:00,15287.61,15330.00,15179.00,15306.83,3371.00,2020-11-10 13:59:59.999,51393461.99,52277,1412.22,21534514.11,0,48.37,41.52


In [15]:
df[['# openTime', 'open', 'high', 'low', 'close', 'volume', 'RSI_10',
       'MFI_10']].tail(10).round(2)

,# openTime,open,high,low,close,volume,RSI_10,MFI_10
490,2020-11-10 05:00:00,15387.74,15441.18,15308.21,15357.92,2178.82,51.56,51.08
491,2020-11-10 06:00:00,15357.92,15390.18,15247.21,15275.82,2558.92,45.66,37.47
492,2020-11-10 07:00:00,15275.83,15333.49,15250.47,15274.74,1981.50,45.58,38.69
493,2020-11-10 08:00:00,15274.75,15460.00,15261.01,15371.37,3795.22,53.33,51.82
494,2020-11-10 09:00:00,15371.36,15440.00,15367.40,15383.84,2253.77,54.27,52.12
495,2020-11-10 10:00:00,15383.84,15430.00,15302.58,15305.14,2365.38,47.59,52.94
496,2020-11-10 11:00:00,15305.15,15323.67,15155.00,15279.24,4014.34,45.54,40.16
497,2020-11-10 12:00:00,15279.24,15393.65,15213.00,15287.61,3484.42,46.37,43.31
498,2020-11-10 13:00:00,15287.61,15330.00,15179.00,15306.83,3371.00,48.37,41.52
499,2020-11-10 14:00:00,15306.84,15308.04,15238.01,15239.49,882.07,42.22,35.54


Visuele controle van de waarden in Tradingview geven aan dat waardes kloppen maar de tijd 1 uur verschilt t.o.v. UTC+1 time.

## Kolommen controleren oversold en overbought condities

Elke vergelijking krijgt een uitkomst mee:

Indien oversold:
* Als 'RSI10 < 32' = 1
* Als 'MFI10 < 19' = 1

Indien beide indicatoren op 1 staan is de waarde in de eindkolom 2 en daarmee een signaal om op te letten voor long momenten.

Indien overbought:
* Als 'RSI > 75' = 3
* Als 'MFI > 85' = 3

Indien beide indicatoren op 3 staan is de waarde in de eindkolom 6 en daarmee een signaal om op te letten voor short momenten.

In [65]:
# SMA check
# RSI check
def x(row):
    if row['RSI_10'] < 32:
        val = 1
    elif row['RSI_10'] > 75:
        val = 3
    else:
        val = 0
    return val
df['RSI_pos'] = df.apply(x, axis=1)
# df['RSI_pos'] = np.where(df['RSI_10']<32, 1 , 0)
# df['RSI_pos'] = np.where(df['RSI_10']>75, 3 , 0)

# MFI check
def y(row):
    if row['MFI_10'] < 21:
        val = 1
    elif row['MFI_10'] > 85:
        val = 3
    else:
        val = 0
    return val
df['MFI_pos'] = df.apply(y, axis=1)

# df['MFI_pos'] = np.where(df['MFI_10']<21, 1 , 0)
# df['MFI_pos'] = np.where(df['MFI_10']>85, 3 , 0)


In [67]:
df[['# openTime','close','RSI_10','RSI_pos','MFI_10','MFI_pos']].tail(30).round(2)

,# openTime,close,RSI_10,RSI_pos,MFI_10,MFI_pos
470,2020-11-09 09:00:00,15404.26,52.18,0,35.45,0
471,2020-11-09 10:00:00,15465.83,56.64,0,35.35,0
472,2020-11-09 11:00:00,15802.39,72.31,0,46.09,0
473,2020-11-09 12:00:00,15565.27,56.36,0,36.73,0
474,2020-11-09 13:00:00,15278.58,43.48,0,30.41,0
475,2020-11-09 14:00:00,15101.00,37.57,0,27.49,0
476,2020-11-09 15:00:00,15000.96,34.63,0,20.98,1
477,2020-11-09 16:00:00,15111.62,40.37,0,18.94,1
478,2020-11-09 17:00:00,15225.00,45.80,0,24.84,0
479,2020-11-09 18:00:00,15317.51,49.93,0,31.19,0


Nu een kolom voor de telling van de posities en een kolom die obv. deze telling een advies geeft.

In [68]:
# Posities tellen
df['POS_cnt'] = df['RSI_pos'] + df['MFI_pos']
df[['RSI_pos', 'MFI_pos', 'POS_cnt']].tail(15)

,RSI_pos,MFI_pos,POS_cnt
485,0,0,0
486,0,0,0
487,0,0,0
488,0,0,0
489,0,0,0
490,0,0,0
491,0,0,0
492,0,0,0
493,0,0,0
494,0,0,0


In [69]:
df.loc[df['POS_cnt'] == 2, ['# openTime','close']]

,# openTime,close
185,2020-10-28 12:00:00,13293.51
186,2020-10-28 13:00:00,13106.31
187,2020-10-28 14:00:00,13069.26
189,2020-10-28 16:00:00,13142.02
304,2020-11-02 11:00:00,13402.18
305,2020-11-02 12:00:00,13297.56
431,2020-11-07 18:00:00,14927.95
432,2020-11-07 19:00:00,14643.52
433,2020-11-07 20:00:00,14852.26
434,2020-11-07 21:00:00,14822.58


In [70]:
df.loc[df['POS_cnt'] == 6, ['# openTime','close']]

,# openTime,close
11,2020-10-21 06:00:00,12232.76
20,2020-10-21 15:00:00,12735.00
21,2020-10-21 16:00:00,12824.35
22,2020-10-21 17:00:00,12760.17
23,2020-10-21 18:00:00,12761.61
372,2020-11-05 07:00:00,14475.87
373,2020-11-05 08:00:00,14444.09
375,2020-11-05 10:00:00,14648.00
376,2020-11-05 11:00:00,14717.60
377,2020-11-05 12:00:00,14870.14


In [95]:
# Advies geven
def f(row):
    if row['POS_cnt'] == 0:
        val = 'NEUTRAL'
    elif row['POS_cnt'] == 2:
        val = 'BUY'
    elif row['POS_cnt'] == 6:
        val = 'SELL'
    else:
        val = 'NEUTRAL'
    return val

#create new column 'POS-adv' using the function above
df['POS_adv'] = df.apply(f, axis=1)

In [96]:
df[['# openTime','close','POS_cnt','POS_adv']].tail(25)

,# openTime,close,POS_cnt,POS_adv
475,2020-11-09 14:00:00,15101.00,0,NEUTRAL
476,2020-11-09 15:00:00,15000.96,1,NEUTRAL
477,2020-11-09 16:00:00,15111.62,1,NEUTRAL
478,2020-11-09 17:00:00,15225.00,0,NEUTRAL
479,2020-11-09 18:00:00,15317.51,0,NEUTRAL
480,2020-11-09 19:00:00,15360.00,0,NEUTRAL
481,2020-11-09 20:00:00,15373.76,0,NEUTRAL
482,2020-11-09 21:00:00,15369.24,0,NEUTRAL
483,2020-11-09 22:00:00,15242.61,0,NEUTRAL
484,2020-11-09 23:00:00,15328.41,0,NEUTRAL


In [77]:
df.loc[df['POS_adv'] == 'BUY', ['POS_adv','# openTime','close', 'MFI_10', 'RSI_10']].tail(200).round(2)

,POS_adv,# openTime,close,MFI_10,RSI_10
185,BUY,2020-10-28 12:00:00,13293.51,11.34,27.01
186,BUY,2020-10-28 13:00:00,13106.31,10.60,20.44
187,BUY,2020-10-28 14:00:00,13069.26,9.06,19.40
189,BUY,2020-10-28 16:00:00,13142.02,13.90,31.16
304,BUY,2020-11-02 11:00:00,13402.18,10.69,29.39
305,BUY,2020-11-02 12:00:00,13297.56,8.88,25.06
431,BUY,2020-11-07 18:00:00,14927.95,13.25,20.84
432,BUY,2020-11-07 19:00:00,14643.52,7.12,15.34
433,BUY,2020-11-07 20:00:00,14852.26,3.11,30.35
434,BUY,2020-11-07 21:00:00,14822.58,6.96,29.52


In [78]:
df.loc[df['POS_adv'] == 'SELL', ['POS_adv','# openTime','close', 'MFI_10', 'RSI_10']].tail(200).round(2)

,POS_adv,# openTime,close,MFI_10,RSI_10
11,SELL,2020-10-21 06:00:00,12232.76,85.02,81.88
20,SELL,2020-10-21 15:00:00,12735.00,86.71,90.13
21,SELL,2020-10-21 16:00:00,12824.35,88.31,91.49
22,SELL,2020-10-21 17:00:00,12760.17,86.80,82.43
23,SELL,2020-10-21 18:00:00,12761.61,85.72,82.48
372,SELL,2020-11-05 07:00:00,14475.87,86.85,81.71
373,SELL,2020-11-05 08:00:00,14444.09,87.19,76.97
375,SELL,2020-11-05 10:00:00,14648.00,85.38,84.18
376,SELL,2020-11-05 11:00:00,14717.60,87.44,85.87
377,SELL,2020-11-05 12:00:00,14870.14,88.62,88.79


Print nu alleen de waarde van de positie die ingenomen dient te worden van de laatste volle dag = 498 (regel 499 is altijd van de lopende dag en derhalve nog niet afgesloten).

In [79]:
print (df[['# openTime','POS_adv']].iloc[-2])

# openTime    2020-11-10 13:00:00
POS_adv                  NEUTRAAL
Name: 498, dtype: object


In [80]:
advice = df.iloc[-2]['POS_adv']
date = df.iloc[-2]['# openTime']
print(date)
print(advice)

2020-11-10 13:00:00
NEUTRAAL


## ATR vervanger

In plaats van een ATR te gebruiken voor stop-loss posities, is het voor deze strategie handiger om de laagste candle te pakken en een stop-loss daaronder te plaatsen. Echter omdat het in deze strategie nog kan voorkomen dat de prijs lager kan zakken, is dat handiger om met het oog te bepalen wat na een signaal de koers nog gaat doen.

## Toevoegen van ATR indicator

In [82]:
df.ta.atr(length=14, append=True)

df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'RSI_10', 'MFI_10', 'RSI_pos',
       'MFI_pos', 'POS_cnt', 'POS_adv', 'ATR_14'],
      dtype='object')

In [83]:
df[['# openTime', 'ATR_14']].tail(5)

,# openTime,ATR_14
495,2020-11-10 10:00:00,159.098814
496,2020-11-10 11:00:00,159.782470
497,2020-11-10 12:00:00,161.273008
498,2020-11-10 13:00:00,160.539221
499,2020-11-10 14:00:00,154.074277


ATR bedragen kloppen met de Tradingview chart!

In [84]:
# Adding ATR and SL/TP1 levels
df['Close+atr']=df['close'] + df['ATR_14']
df['Close-atr']=df['close'] - df['ATR_14']
df[['# openTime','close','ATR_14','Close+atr','Close-atr']].tail(10).round(2)

,# openTime,close,ATR_14,Close+atr,Close-atr
490,2020-11-10 05:00:00,15357.92,174.67,15532.59,15183.25
491,2020-11-10 06:00:00,15275.82,172.41,15448.23,15103.41
492,2020-11-10 07:00:00,15274.74,166.02,15440.76,15108.72
493,2020-11-10 08:00:00,15371.37,168.38,15539.75,15202.99
494,2020-11-10 09:00:00,15383.84,161.54,15545.38,15222.30
495,2020-11-10 10:00:00,15305.14,159.10,15464.24,15146.04
496,2020-11-10 11:00:00,15279.24,159.78,15439.02,15119.46
497,2020-11-10 12:00:00,15287.61,161.27,15448.88,15126.34
498,2020-11-10 13:00:00,15306.83,160.54,15467.37,15146.29
499,2020-11-10 14:00:00,15239.49,154.07,15393.56,15085.42


# Alle kolommen

In [87]:
df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'RSI_10', 'MFI_10', 'RSI_pos',
       'MFI_pos', 'POS_cnt', 'POS_adv', 'ATR_14', 'Close+atr', 'Close-atr'],
      dtype='object')

In [86]:
df[['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
        'RSI_10','MFI_10','RSI_pos', 'MFI_pos','POS_cnt', 'POS_adv', 'ATR_14', 'Close+atr',
       'Close-atr']].tail(10).round(2)

,# openTime,open,high,low,close,volume,closeTime,RSI_10,MFI_10,RSI_pos,MFI_pos,POS_cnt,POS_adv,ATR_14,Close+atr,Close-atr
490,2020-11-10 05:00:00,15387.74,15441.18,15308.21,15357.92,2178.82,2020-11-10 05:59:59.999,51.56,51.08,0,0,0,NEUTRAAL,174.67,15532.59,15183.25
491,2020-11-10 06:00:00,15357.92,15390.18,15247.21,15275.82,2558.92,2020-11-10 06:59:59.999,45.66,37.47,0,0,0,NEUTRAAL,172.41,15448.23,15103.41
492,2020-11-10 07:00:00,15275.83,15333.49,15250.47,15274.74,1981.50,2020-11-10 07:59:59.999,45.58,38.69,0,0,0,NEUTRAAL,166.02,15440.76,15108.72
493,2020-11-10 08:00:00,15274.75,15460.00,15261.01,15371.37,3795.22,2020-11-10 08:59:59.999,53.33,51.82,0,0,0,NEUTRAAL,168.38,15539.75,15202.99
494,2020-11-10 09:00:00,15371.36,15440.00,15367.40,15383.84,2253.77,2020-11-10 09:59:59.999,54.27,52.12,0,0,0,NEUTRAAL,161.54,15545.38,15222.30
495,2020-11-10 10:00:00,15383.84,15430.00,15302.58,15305.14,2365.38,2020-11-10 10:59:59.999,47.59,52.94,0,0,0,NEUTRAAL,159.10,15464.24,15146.04
496,2020-11-10 11:00:00,15305.15,15323.67,15155.00,15279.24,4014.34,2020-11-10 11:59:59.999,45.54,40.16,0,0,0,NEUTRAAL,159.78,15439.02,15119.46
497,2020-11-10 12:00:00,15279.24,15393.65,15213.00,15287.61,3484.42,2020-11-10 12:59:59.999,46.37,43.31,0,0,0,NEUTRAAL,161.27,15448.88,15126.34
498,2020-11-10 13:00:00,15287.61,15330.00,15179.00,15306.83,3371.00,2020-11-10 13:59:59.999,48.37,41.52,0,0,0,NEUTRAAL,160.54,15467.37,15146.29
499,2020-11-10 14:00:00,15306.84,15308.04,15238.01,15239.49,882.07,2020-11-10 14:59:59.999,42.22,35.54,0,0,0,NEUTRAAL,154.07,15393.56,15085.42


## Nieuw idee: Check in Panda's of een veldwaarde anders is t.o.v. de vorige waarde

Dus als veld 499 != 498 , dan 1 anders 0.  <
Als 1 dan waarschuwing via signals

In [88]:
df.iloc[-2]['POS_adv']

'NEUTRAAL'

In [89]:
df['POS_adv_changed'] = df['POS_adv'].shift(+1)==df['POS_adv']
df[['POS_adv_changed','POS_adv','# openTime']].tail(30)

,POS_adv_changed,POS_adv,# openTime
470,True,NEUTRAAL,2020-11-09 09:00:00
471,True,NEUTRAAL,2020-11-09 10:00:00
472,True,NEUTRAAL,2020-11-09 11:00:00
473,True,NEUTRAAL,2020-11-09 12:00:00
474,True,NEUTRAAL,2020-11-09 13:00:00
475,True,NEUTRAAL,2020-11-09 14:00:00
476,True,NEUTRAAL,2020-11-09 15:00:00
477,True,NEUTRAAL,2020-11-09 16:00:00
478,True,NEUTRAAL,2020-11-09 17:00:00
479,True,NEUTRAAL,2020-11-09 18:00:00


In [94]:
df.loc[df.POS_adv_changed== False , ['# openTime','POS_adv_changed','POS_adv']].tail(25)

,# openTime,POS_adv_changed,POS_adv
0,2020-10-20 19:00:00,False,NEUTRAAL
11,2020-10-21 06:00:00,False,SELL
12,2020-10-21 07:00:00,False,NEUTRAAL
20,2020-10-21 15:00:00,False,SELL
24,2020-10-21 19:00:00,False,NEUTRAAL
185,2020-10-28 12:00:00,False,BUY
188,2020-10-28 15:00:00,False,NEUTRAAL
189,2020-10-28 16:00:00,False,BUY
190,2020-10-28 17:00:00,False,NEUTRAAL
304,2020-11-02 11:00:00,False,BUY


In [91]:
df

,# openTime,open,high,low,close,volume,closeTime,qouteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,...,RSI_10,MFI_10,RSI_pos,MFI_pos,POS_cnt,POS_adv,ATR_14,Close+atr,Close-atr,POS_adv_changed
0,2020-10-20 19:00:00,11964.58,11981.20,11881.00,11912.35,2744.144578,2020-10-20 19:59:59.999,3.274267e+07,39342,1126.575869,...,NaN,NaN,0,0,0,NEUTRAAL,100.200000,12012.550000,11812.150000,False
1,2020-10-20 20:00:00,11912.42,11912.79,11855.00,11897.92,2523.731342,2020-10-20 20:59:59.999,2.998975e+07,39583,1251.684940,...,0.000000,NaN,1,0,1,NEUTRAAL,78.209630,11976.129630,11819.710370,True
2,2020-10-20 21:00:00,11897.92,11937.73,11877.49,11922.53,1409.942909,2020-10-20 21:59:59.999,1.679021e+07,28867,642.188103,...,65.457350,NaN,0,0,0,NEUTRAAL,71.770786,11994.300786,11850.759214,True
3,2020-10-20 22:00:00,11922.53,11961.00,11918.22,11950.66,1270.092250,2020-10-20 22:59:59.999,1.516067e+07,27532,603.063086,...,81.137955,NaN,3,0,3,NEUTRAAL,63.698669,12014.358669,11886.961331,True
4,2020-10-20 23:00:00,11950.66,11961.16,11885.00,11909.99,1551.691786,2020-10-20 23:59:59.999,1.850040e+07,24948,757.031464,...,46.921228,NaN,0,0,0,NEUTRAAL,66.573296,11976.563296,11843.416704,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-11-10 10:00:00,15383.84,15430.00,15302.58,15305.14,2365.383922,2020-11-10 10:59:59.999,3.637861e+07,48025,1116.957106,...,47.587632,52.938945,0,0,0,NEUTRAAL,159.098814,15464.238814,15146.041186,True
496,2020-11-10 11:00:00,15305.15,15323.67,15155.00,15279.24,4014.340547,2020-11-10 11:59:59.999,6.121471e+07,65654,1954.016547,...,45.537890,40.161409,0,0,0,NEUTRAAL,159.782470,15439.022470,15119.457530,True
497,2020-11-10 12:00:00,15279.24,15393.65,15213.00,15287.61,3484.423100,2020-11-10 12:59:59.999,5.331060e+07,56464,1696.465821,...,46.367392,43.305521,0,0,0,NEUTRAAL,161.273008,15448.883008,15126.336992,True
498,2020-11-10 13:00:00,15287.61,15330.00,15179.00,15306.83,3370.999279,2020-11-10 13:59:59.999,5.139346e+07,52277,1412.224206,...,48.373617,41.520391,0,0,0,NEUTRAAL,160.539221,15467.369221,15146.290779,True
